## Import & 전역변수


In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pickle

MOVIE_LIST_PATH = '../../data_analysis/movie/movie.pickle'
VISIT_PATH = './visit.pickle'
MOVIE_POSTER_PATH = './movie_poster/'
BASE_URL = 'https://kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do?dtTp=movie&dtCd='
BASE_RESULT_URL = 'https://kobis.or.kr'
# HOW_MANY = 900 # 3600초 기대
HOW_MANY = 110
CHECK_POINT = 100

## 체크리스트 만들기 (VISIT, 초기 한번만 실행)

In [6]:
with open(MOVIE_LIST_PATH, 'rb') as f :
    MOVIE_LIST = pickle.load(f)
VISIT = {movie_code : False for movie_code in MOVIE_LIST}
print(f"Num of movies : {len(VISIT)}")
with open(VISIT_PATH, 'wb') as f :
    pickle.dump(VISIT, f)


Num of movies : 82557


## 영화진흥위원회 사이트에서 영화 포스터 크롤링 하기


In [20]:
start_time = time.time()
with open(VISIT_PATH, 'rb') as f :
    VISIT = pickle.load(f)

file_count = 0
for movie_code, is_visit in VISIT.items() :
    if is_visit :
        continue

    if file_count != 0 and file_count % CHECK_POINT == 0 :
        print(f"Now file_count : {file_count}")
        print(f"Now execution time : {time.time()-start_time}")

    try :
        url = BASE_URL+movie_code
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=options)
        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        img_src = soup.find("a", {'class':'fl thumb'})

        target_url = img_src['href']
        result_url = BASE_RESULT_URL+target_url

        result = {movie_code:result_url}
        with open(MOVIE_POSTER_PATH+f"{movie_code}_poster.pickle", 'wb') as f :
            pickle.dump(result, f)

        VISIT[movie_code] = True
        file_count += 1
    except :
        continue

    # if file_count == HOW_MANY :
    #         break

with open(VISIT_PATH, 'wb') as f :
    pickle.dump(VISIT, f)

print(f"Completed file count : {file_count}")
print(f"Execution time : {time.time()-start_time}")


Completed file count : 18
Execution time : 71.0552589893341
